Importações

In [6]:
import time
import scipy.linalg as sp
import numpy as np
import sympy as sym
from math import *


TODO: 3o item
      comparacoes/analises
      talvez mudar tudo para não usar sympy

## ITEM 1

In [20]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
f1 = x1*x2 +x1 -3*x5
f2 = 2*x1*x2 +x1 +3*R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 -R[0]*x5
f3 = 2*x2*(x3**2) +R[-4]*x2*x3 + 2*R[-6]*(x3**2) +R[-5]*x3 -8*x5
f4 = R[-2]*x2*x4 +2*(x4**2) -4*R[0]*x5
f5 = x1*x2 +x1 +R[-1]*(x2**2) +x2*(x3**2) +R[-4]*x2*x3 +R[-2]*x2*x4 +R[-3]*x2 +R[-6]*(x3**2) +R[-5]*x3 +(x4**2) -1
    
F = np.transpose(np.array([f1,f2,f3,f4,f5]))

In [21]:
## calculate values of F under a vector x
def calcF(F, x):
    Fx = []
    for f in F:
        Fx.append(f.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
    return np.array(Fx)

## define jacobian matrix
def J(F):
    Jmatrix = []
    for f in F:
        Jmatrix.append([f.diff(x1), f.diff(x2), f.diff(x3), f.diff(x4), f.diff(x5)])
        
    return np.array(Jmatrix)    


## calculate values of J under a vector x
def calcJ(J, x):
    Jx = []
    i = 0
    for linha in J:
        Jx.append([])
        for j in linha:
            Jx[i].append(j.subs([(x1,x[0]),(x2,x[1]),(x3,x[2]),(x4,x[3]),(x5,x[4])]))
        
        i += 1
        
        
    return np.array(Jx)

## NOTE: the functions J(), calcJ() and calcF() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.



### Método de Newton

In [8]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F): # INPUT
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
   
    # Step 1
    k = 1 
    print("Starting with Xo = ", x)
    
    # Step 2
    while k <= maxIterations:
        
        # Step 3
        Fx = [-x for x in calcF(F,x)] # here, -Fx was calculated, for easier use in next step
        Jx = calcJ(J(F), x)
        
        # Step 4
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(Fx, np.float64))
        
        # Step 5
        x = x + y
        print(k,"th iteration x =", x, end=" ")
        
        # Step 6
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norm: ", np.linalg.norm(x - (x-y), 2))
            break #STOP
        else:
            print("Norm: ", np.linalg.norm(x - (x-y), 2))
            
            # Step 7
            k += 1
    
    # Step 8
    return output

In [10]:
# solve with Newton's method

x = [10, 10, 10, 10, 10]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-6) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 20 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodNewton(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Time to calculate: {t1} seconds")


Starting with Xo =  [10 10 10 10 10]
1 th iteration x = [0.01065204 9.99922036 5.00102591 5.03641579 0.03645869] Norm:  15.769785630785213
2 th iteration x = [ 0.01001153 10.01012371  2.49920776  2.59112275  0.03674507] Norm:  3.498381268945265
3 th iteration x = [9.98373148e-03 1.00839478e+01 1.24338306e+00 1.43787284e+00
 3.68870704e-02] Norm:  1.7066139146402113
4 th iteration x = [9.68920699e-03 1.04317749e+01 6.06359646e-01 9.75892034e-01
 3.69557589e-02] Norm:  0.8603539169025427
5 th iteration x = [8.50591251e-03 1.18477258e+01 2.74174277e-01 8.66848855e-01
 3.69857062e-02] Norm:  1.4584772627650562
6 th iteration x = [5.41646124e-03 1.67133282e+01 1.03992448e-01 8.60022364e-01
 3.69918658e-02] Norm:  4.868583458334746
7 th iteration x = [2.92103470e-03 2.76408171e+01 6.02819450e-02 8.59724155e-01
 3.69765221e-02] Norm:  10.927576589262472
8 th iteration x = [3.45584277e-03 3.17170310e+01 6.96040198e-02 8.59516377e-01
 3.69616409e-02] Norm:  4.07622454194743
9 th iteration x = [

### Método de Broyden

In [11]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F): # INPUT
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    
    # Step 1
    Ao = calcJ(J(F), x)
    v = calcF(F, x)
    
    # Step 2
    A = np.linalg.inv(np.array(Ao, dtype=np.float64))
    
    # Step 3
    s = np.dot(-A,v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    
    # Step 4
    while k <= maxIterations:
        
        # Step 5
        w = v
        v = calcF(F, x)
        y = v - w
        
        # Step 6
        z = -np.dot(A,y)
        
        # Step 7
        p = np.dot(-np.transpose(s),z)
        
        # Step 8
        ut = np.dot(np.transpose(s),A)
        
        # Step 9
        A = A + (np.outer((s+z),ut))/p
        
        # Step 10
        s = -np.dot(A,v)
        
        # Step 11
        x = x + s
        print(k,"th iteration x=", x, end=" ")
        
        # Step 12
        if np.linalg.norm(np.array(s, np.float64), 2) < tolerance:
            print("Norm: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break #STOP
        print("Norm: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        
        # Step 13
        k += 1
    
    # Step 14
    return output
             
    

In [22]:
# solve with Broyden's method

x = [1, 30, 1, 1, 1] # para [1, 30, 1, 1, 1] converge na 16!!
Xo = np.transpose(np.array(x)) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-6) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 1000 # so it doesnt loop forever in case of failure

t0 = time.time() # measure time it takes to calculate
print(methodBroyden(n, Xo, tolerance, maxIterations, F))
t1 = time.time() - t0
print(f"Time to calculate: {t1} seconds")

Começando com Xo =  [ 1 30  1  1  1]
X1 =  [0.00606465637549924 29.9224936866024 0.503653761479361 0.868105595566895
 0.0368326780809132]
2 th iteration x= [0.00508894473219099 29.8733912939840 0.376429180191370 0.859534236017904
 0.0368725659704363] Norm:  109.04634624620046
3 th iteration x= [0.00403355871612505 29.7966483862168 0.216595784546808 0.857642380131586
 0.0369222799403499] Norm:  8.314918711098795
4 th iteration x= [0.00361578324136782 29.7624149031520 0.146038362157734 0.858973009543936
 0.0369439554645721] Norm:  7.028350430491454
5 th iteration x= [0.00340313999509602 29.7531637094191 0.111660869281349 0.859458953344956
 0.0369544577230729] Norm:  1.896447767762025
6 th iteration x= [0.00334704384466043 29.7597331342939 0.102721450283580 0.859477681056703
 0.0369571559860851] Norm:  0.6584095262238857
7 th iteration x= [0.00333479970475373 29.7795042301823 0.100196401975186 0.859455111917542
 0.0369578560229294] Norm:  0.14261518561960956
8 th iteration x= [0.003347279

### Comparação dos resultados

In [18]:
from scipy.optimize import broyden1

# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
def f1(x):
    return (x[0]*x[1]) + x[0] -(3*x[4])

def f2(x):
    global R
    return 2*x[0]*x[1] +x[0] +3*R[-1]*(x[1]**2) +x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] -R[0]*x[4]

def f3(x):
    global R
    return 2*x[1]*(x[2]**2) +R[-4]*x[1]*x[2] + 2*R[-6]*(x[2]**2) +R[-5]*x[2] -8*x[4]

def f4(x):
    global R
    return R[-2]*x[1]*x[3] +2*(x[3]**2) -4*R[0]*x[4]

def f5(x):
    global R
    return x[0]*x[1] + x[0] +R[-1]*(x[1]**2) + x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] +R[-6]*(x[2]**2) +R[-5]*x[2] +(x[3]**2) -1

Fmatrix = [f1, f2, f3, f4, f5]
    
def F(x):
    return np.transpose(np.array([f1(x),f2(x),f3(x),f4(x),f5(x)], np.float64))

broyden1(F, x)

NoConvergence: [-15.58762779  -1.06582507   3.02460955   4.03635482   0.28225442]

Ambos os métodos são feitos para inicair de um ponto que já tenha uma aproximação relativamente boa. O método de Newton apresentou uma boa convergência, enquanto o de Broyden não convergiu. De fato, é possível perceber que em outras implementações do método de Broyden, com o ponto inicial dado para as funções dadas, a função não converge.

garantir que para newton f'(pi) '= 0

## ITEM 2

In [3]:
# define the functions
x1, x2, x3, x4, x5 = sym.symbols('x1 x2 x3 x4 x5')
R = [10, .193, 4.10622*10**(-4), 5.45177*10**(-4), 4.4975*10**(-7), 3.40735*10**(-5), 9.615*10**(-7)]

## define matrix F
def f1(x):
    return (x[0]*x[1]) + x[0] -(3*x[4])

def f2(x):
    global R
    return 2*x[0]*x[1] +x[0] +3*R[-1]*(x[1]**2) +x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] -R[0]*x[4]

def f3(x):
    global R
    return 2*x[1]*(x[2]**2) +R[-4]*x[1]*x[2] + 2*R[-6]*(x[2]**2) +R[-5]*x[2] -8*x[4]

def f4(x):
    global R
    return R[-2]*x[1]*x[3] +2*(x[3]**2) -4*R[0]*x[4]

def f5(x):
    global R
    return x[0]*x[1] + x[0] +R[-1]*(x[1]**2) + x[1]*(x[2]**2) +R[-4]*x[1]*x[2] +R[-2]*x[1]*x[3] +R[-3]*x[1] +R[-6]*(x[2]**2) +R[-5]*x[2] +(x[3]**2) -1

Fmatrix = [f1, f2, f3, f4, f5]
    
def F(x):
    return np.transpose(np.array([f1(x),f2(x),f3(x),f4(x),f5(x)], np.float64))


In [4]:
# parcial derivatives by finite differences - FIRST ORDER
def df1(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/h
    
    return df_xk


# parcial derivatives by finite differences - SECOND ORDER
def df2(f, h, x, k): # function, step, x vector, k direction of derivative
    # create step in k direction
    x0 = np.array(x, np.float64)
    x1 = np.array(x, np.float64)
    x0[k] -= h
    x1[k] += h
     
    df_xk = (f(x1) - f(x0))/(2*h)
    
    return df_xk


## define jacobian matrix
def J(Fmatrix, x, order, h):
    Jmatrix = []
    
    if order == 1:
        for f in Fmatrix:
            Jmatrix.append([df1(f, h, x, 0), df1(f, h, x, 1), df1(f, h, x, 2), df1(f, h, x, 3), df1(f, h, x, 4)])
    elif order == 2:
        for f in Fmatrix:
            Jmatrix.append([df2(f, h, x, 0), df2(f, h, x, 1), df2(f, h, x, 2), df2(f, h, x, 3), df2(f, h, x, 4)])
        
        
    return np.array(Jmatrix, np.float64)    



## NOTE: the function J() could've been done in a more general way, such as to be used for any size of linear system
## adjusting automatically, but since we're just using this program for specific examples, for simplicity I used one for each example.
## Feel free to update it to work in a more general way.




### Método de Newton

In [34]:
# define method
def methodNewton(n, x, tolerance, maxIterations, F, Fmatrix, h):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    # Step 1
    k = 1
    print("Começando com Xo = ", x)
    
    # Step 2
    while k <= maxIterations:
        
        # Step 3
        Jx = J(Fmatrix, x, 2, h)
        
        # Step 4
        y = np.linalg.solve(np.array(Jx, np.float64), np.array(-F(x), np.float64))
        
        # Step 5
        x = x + y
        print(k," Iteração x=", x, end=" ")
        
        # Step 6
        if np.linalg.norm(y, np.inf)  < tolerance:
            output = x
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            break #STOP
        else:
            print("Norma: ", np.linalg.norm(x - (x-y), 2))
            
            # Step 7
            k += 1
        
        
    # Step 8
    return output

In [35]:
# solve with Newton's method

x = [1, 30, 1, 1, 1]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-6) # chosen - arbitrary because there's no real implication to have a specific tolerance defined
maxIterations = 1000 # so it doesnt loop forever in case of failure
h = 10000

t0 = time.time() # measure time it takes to calculate
print(methodNewton(n, Xo, tolerance, maxIterations, F, Fmatrix, h))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [ 1 30  1  1  1]
1  Iteração x= [6.06465638e-03 2.99224937e+01 5.03653761e-01 8.68105596e-01
 3.68326781e-02] Norma:  1.4782968945230994
2  Iteração x= [3.67451026e-03 2.94458435e+01 2.60610053e-01 8.58999844e-01
 3.69114336e-02] Norma:  0.5351208054287151
3  Iteração x= [3.62159315e-03 2.96028508e+01 1.39106350e-01 8.59357869e-01
 3.69464611e-02] Norma:  0.19853105959525252
4  Iteração x= [3.50026913e-03 3.06407697e+01 8.48748336e-02 8.59495340e-01
 3.69590445e-02] Norma:  1.0393346889799222
5  Iteração x= [3.43596181e-03 3.12607127e+01 6.97150519e-02 8.59526145e-01
 3.69621480e-02] Norma:  0.6201284216539408
6  Iteração x= [3.43027445e-03 3.13259690e+01 6.83614888e-02 8.59528972e-01
 3.69624389e-02] Norma:  0.06527026630739176
7  Iteração x= [3.43023016e-03 3.13264968e+01 6.83504021e-02 8.59528996e-01
 3.69624414e-02] Norma:  0.000527908851007378
8  Iteração x= [3.43023016e-03 3.13264968e+01 6.83504014e-02 8.59528996e-01
 3.69624414e-02] Norma:  3.497985342485894e

### Método de Broyden

In [26]:
# define method
def methodBroyden(n, x, tolerance, maxIterations, F, Fmatrix, h): # n is the number of equations and unknowns
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    print("Começando com Xo = ", x)
    
    # Step 1
    Ao = J(Fmatrix, x, 1, h)
    v = F(x)
    
    # Step 2
    A = np.linalg.inv(np.array(Ao, dtype=np.float64))
   
    # Step 3
    s = np.dot(-A,v)
    x = x + s
    k = 2
    
    print("X1 = ", x)
    
    # Step 4
    while k <= maxIterations:
        
        # Step 5
        w = v
        v = F(x)
        y = v - w
        
        # Step 6
        z = -np.dot(A,y)
        
        # Step 7
        p = np.dot(-np.transpose(s),z)
        
        # Step 8
        ut = np.dot(np.transpose(s),A)
        
        # Step 9
        A = A + (np.outer((s+z),ut))/p
        
        # Step 10
        s = -np.dot(A,v)
        
        # Step 11
        x = x + s
        print(k," Iteração x=", x, end=" ")
        
        # Step 12
        if np.linalg.norm(np.array(s, np.float64), 2) < tolerance:
            print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
            output = x
            break #STOP
        print("Norma: ", sp.norm(np.array(x - (x-y),dtype=np.float64), 2))
        
        # Step 13
        k += 1
        
    # Step 14
    return output

In [31]:
# solve with Broyden's method

x = [1, 30, 1, 1, 1] # também convergiu em 16 iterações na [1, 30, 1, 1, 1]
Xo = np.transpose(x) # initial approximation
n = 5 # number of equations and unknowns
tolerance = 10**(-6) 
maxIterations = 1000 # so it doesnt loop forever in case of failure
h = 100

t0 = time.time() # measure time it takes to calculate
print(methodBroyden(n, Xo, tolerance, maxIterations, F, Fmatrix, h))
t1 = time.time() - t0
print(f"Tempo para calcular: {t1} segundos")

Começando com Xo =  [ 1 30  1  1  1]
X1 =  [9.35112329e-03 2.98201585e+01 9.90299437e-01 9.97383482e-01
 3.66810956e-02]
2  Iteração x= [8.28871478e-03 2.96724044e+01 9.80699261e-01 9.94782559e-01
 3.66858858e-02] Norma:  77.10427611281268
3  Iteração x= [3.76865076e-03 2.90232410e+01 9.30215936e-01 9.81060042e-01
 3.67093941e-02] Norma:  1.7828442430877696
4  Iteração x= [3.65971277e-03 2.89341949e+01 9.15239622e-01 9.76961165e-01
 3.67147302e-02] Norma:  8.61493335822048
5  Iteração x= [3.66326975e-03 2.89155553e+01 9.04120092e-01 9.73911381e-01
 3.67182861e-02] Norma:  2.166714352809265
6  Iteração x= [3.66751516e-03 2.88997639e+01 8.85162174e-01 9.68719997e-01
 3.67242397e-02] Norma:  1.4819022845053849
7  Iteração x= [3.71429629e-03 2.86796269e+01 6.15792060e-01 8.95124190e-01
 3.68087704e-02] Norma:  2.450971570598549
8  Iteração x= [3.59734819e-03 2.92512322e+01 1.29032565e+00 1.07894706e+00
 3.65978454e-02] Norma:  29.050110413060533
9  Iteração x= [3.72610746e-03 2.86161893e+0

### Comparação dos resultados

comparar normas

## ITEM 3

In [49]:
# define the functions

def f1(x):
    return x[0]**2 + x[2]**2 - 1

def f2(x):
    return x[1]**2 + x[3]**2 - 1

def f3(x):
    return x[4]*(x[2]**3) + x[5]*(x[3]**3)

def f4(x):
    return x[4]*(x[0]**3) + x[5]*(x[1]**3)

def f5(x):
    return x[4]*x[0]*(x[2]**2) + x[5]*x[1]*(x[3]**2)

def f6(x):
    return x[4]*x[2]*(x[0]**2) + x[5]*x[3]*(x[1]**2)

Fmatrix = [f1, f2, f3, f4, f5, f6]

def F(x):
    F1 = f1(x)
    F2 = f2(x)
    F3 = f3(x)
    F4 = f4(x)
    F5 = f5(x)
    F6 = f6(x)
    return np.array([F1, F2, F3, F4, F5, F6], np.float64)

In [50]:
# define the function for Steepest Descent
def g(x, Fmatrix):
    g = 0
    for f in Fmatrix:
        g += (f(x))**2
    return g

# define the method
def SteepestDescent(n, x, tolerance, maxIterations, g, Fmatrix):
    
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    # Step 1
    k = 1
    
    # Step 2
    while k< maxIterations:
        
        # Step 3
        g1 = g(x, Fmatrix)
        z = np.gradient(x)
        z0 = np.linalg.norm(z, 2)
        
        # Step 4
        if z0 == 0:
            print("Zero gradient.\nThe procedure completed, might have a minimum")
            output = x, g1
            break #STOP
        
        # Step 5
        z = z/z0
        alpha1, alpha3 = 0, 1
        g3 = g(x-alpha3*z, Fmatrix)
        
        # Step 6
        while g3>=g1:
            
            # Step 7
            alpha3 /= 2
            g3 = g(x-alpha3*z, Fmatrix)
            
            # Step 8
            if alpha3 < (tolerance/2):
                print("No likely improvement. The procedure completed, might have a minimum")
                output = x, g1
                return output #STOP
            
            # Step 9
            alpha2 = alpha3/2
            g2 =  g(x-alpha2*z, Fmatrix)
            
            # Step 10
            h1 = (g2-g1)/alpha2
            h2 = (g3-g2)/(alpha3-alpha2)
            h3 = (h2-h1)/alpha3
            
            # Step 11
            alpha0 = (alpha2-h1/h3)/2
            g0 = g(x-alpha0*z, Fmatrix)
            
            # Step 12
            if g0 < g3:
                alpha = alpha0
                G = g0
            else:
                alpha = alpha3
                G = g3
            
            # Step 13
            x = x - alpha*z
            
            # Step 14
            if abs(G-g1) < tolerance:
                print("The procedure was successful")
                output = x, G
                return output #STOP
        
        # Step 15
        k += 1
            
    # Step 16
    return output
        

In [52]:
# define method
def fixedPoint(Xo, tolerance, maxIterations, F):
    output = "The method failed after "+str(maxIterations)+" iterations. The procedure was unsuccessful"
    
    # Step 1
    i = 1
    
    # Step 2
    while i <= maxIterations:
        
        # Step 3
        x = F(Xo)
        
        # Step 4
        if np.linalg.norm(x - Xo) < tolerance:
            output = x
            break #STOP
        
        # Step 5
        i += 1
        
        # Step 6
        Xo = x
    
    # Step 7
    return output

In [80]:
# solve with Steepest Descent
def g(x, Fmatrix):
    g = 0
    for f in Fmatrix:
        g += (f(x))**2
    return g

n = 6 # number of equations and unknowns
x = [1, 1, 1, 1, 0, 0] # starting point
tolerance = 10**(-10)
maxIterations = 50

r = SteepestDescent(n, x, tolerance, maxIterations, g, Fmatrix)
print(r)


The procedure was successful
(array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
       -2.05794189e-11,  0.00000000e+00]), 1.9999999999176823)


In [81]:
# solve with fixed point
Xo = np.array(r[0], np.float64)
maxIterations = 1000
tolerance = 0.1

print(r[0])
def g(x):
    return x - F(x)

result = fixedPoint(Xo, tolerance, maxIterations, g)

print(result)
F(result)

[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
 -2.05794189e-11  0.00000000e+00]
[-4.11586321e-11  4.11588541e-11  1.00000000e+00  1.00000000e+00
 -8.47025300e-22  2.05794189e-11]


array([-2.22044605e-16,  0.00000000e+00,  2.05794189e-11,  1.43490429e-42,
        8.47025300e-22,  3.48625908e-32])

#### Bibliografia e referências: 

R.L. Burden & J.D. Faires, Numerical Analysis, Capítulos 2 e 10